Test of a Generalized Metropolis-Hastings MCMC to explore the parameter space of the FitzHugh-Nagumo model: http://www.scholarpedia.org/article/FitzHugh-Nagumo_model 

If the number of proposals per iteration equals 1, then the behaviour of this runner is equivalent to Standard M-H, for number of proposals > 1, it will behave as the Generalized Metropolis-Hastings algorithm.

OPERATION: 
- Run a cell by pressing the black triangle in the toolbar above. 
- Note that the execution of a cell may take a while, and will be confirmed by a printout. 
- If a cell prints output in a pink box, re-run and see if it disappears. If not, close and re-open the notebook, or select "Kernel/Restart" at the top. 
- To remove all printed output and figures, select "Cell/All Output/Clear" at the top.

In [ ]:
###Use 8 processes (1 main and 7 workers) as JuliaBox gives you 8 processors
NPROCS = 8
if nprocs() < NPROCS
    addprocs(NPROCS-nprocs())
end
println("Number of parallel processes: ",nprocs())
    
###The following statement makes the GeneralizedMetropolisHastings core code available on all processes
push!(LOAD_PATH,"../../../src/")
@everywhere using GeneralizedMetropolisHastings
   
###The next function is only available if the GeneralizedMetropolisHastings package has loaded correctly
print_gmh_module_loaded()

###Load the PyPlot package
using PyPlot
println("PyPlot package loaded successfully")

In [ ]:
###Innitialize variables
#Total number of MCMC samples
nburnin = 1000
nsamples = 10000

#Standard M-H for nproposals == 1
#Generalized M-H for nproposals > 1
nproposals = 10

###Initial conditions for the FitzHugh-Nagumo ODE (membrane potential and refractory variable)
initialconditions = [-1.0,1.0]

###Default values of the parameters (a,b,c)
defaultparams = [0.2,0.2,3.0]

###The variance of the noise on the input data
noise = [0.02 0.005]

println("Simulation parameters defined successfully")

The following functions specify the function that holds the measurement data, the Ordinary Differential Equation of the FitzHugh-Nagumo model and a helper function to create the model.

In [ ]:
###Make the data generation function available to the script
include("../data/fitzhughnagumo.jl")

###Make the ode and model creation function available to the script
include("../models/fitzhughnagumo.jl")

println("FitzHugh-Nagumo helper functions defined successfully")

Specify the model object from the parameters and helper function specified above

In [ ]:
###Create a FitzHugh-Nagumo model with measurement data and ODE function
m = fitzhugh_nagumo_model(initialconditions,defaultparams,noise)

figure("FitzHughNagumo-Data")
plot(m.timepoints,m.measurements[:,1];label="membrane potential")
plot(m.timepoints,m.measurements[:,2];label="refractory variable")
xlabel("Time")
ylabel("Amplitude")
title("FitzHugh-Nagumo measurement data")
grid("on")
legend(loc="upper right",fancybox="true")

###Show the model
println("==========================")
println("Model defined successfully")
println("==========================")
show(m)

Now specify the sampler (a Metropolis-Hastings sampler with a Gaussian proposal density) and the runner to run a Generalized Metropolis-Hastings algorithm (remember that the choice between Standard and Generalized M-H is made by either setting nproposals to 1 or make it > 1).

In [ ]:
###Create a Metropolis sampler with a Normal proposal density
s = MHNormal(eye(3),0.01)
show(s)

###Create a Generalized Metropolis-Hastings runner (which will default to Standard MH because nproposals == 1)
###The values of the parameters will be initialized from the default values specified above
r = GMHRunner(nsamples,nproposals;burnin = nburnin,initialize =ValuesFromDefault())
show(r)

Now run the simulation using the runner, the model and the sampler specified above. A printout will appear when the simulation is finished.

In [ ]:

###Run the MCMC (can take quite a bit of time)
c = run!(r,m,s)
###Show the results of the simulations
show(c)
println("Results of the MCMC simulation:")
for i=1:3
  println(" parameter $(m.parameters.names[i]):  mean = ",mean(c.values[i,nburnin:end])," std = ",std(c.values[i,nburnin:end]))
end

In [ ]:
###Plot the loglikelihood values across samples
###After an initial few low values, this should remain relatively high
figure("FitzHugh-Nagumo1-LL")
plot(1:length(c.loglikelihood),c.loglikelihood+c.logprior)
title("Loglikelihood values across samples")
xlabel("Samples")
ylabel("loglikelihood - logprior")

In [ ]:
###Plot the histograms of a,b,c values
for i=1:3
    subplot(220+i)
  h = PyPlot.plt.hist(c.values[i,:]',20)
  grid("on")
  title("Parameter $(m.parameters.names[i])")
  xlabel("Values")
  ylabel("Samples")
end

In [ ]:
println("Number of processes running: ",nprocs())
println("Number of workers running: ",nworkers())
println("Process IDs: ",procs())

In [ ]:
###Only run this box if you want to shut down all worker processes
println("Pre processes running: ",procs())
for i in workers()
    if i != 1
        rmprocs(i)
    end
end
println("Post processes running: ",procs())